In [ ]:
from tiled.client import from_uri, logout_all
from getpass import getpass

import functools
import ipywidgets as widgets
import numpy
import pandas

# 1. Access the online data

## Connect to server

In [ ]:
logout_all() # For demo purposes, ensure logout first

In [ ]:
client = from_uri("https://aimm-staging.lbl.gov/api")

In [ ]:
client.login()

In [ ]:
client

## Browse through all the datasets by code

In [ ]:
client["element"]

In [ ]:
client["element"]["Cr"]

In [ ]:
client["element"]["Cr"]['edge']['K']['uid']['gZbKaqF9EdE'].read()

## Browse through all the datasets with widgets

In [ ]:
class DropDownGroup:
    
    def __init__(self, root_node):
        self.root_node = root_node
        root_list = list(self.root_node)
        root_list.insert(0, "-")
        
        dd_root = widgets.Dropdown(
            options=root_list,
            value=root_list[0],
            description='Node_0:',
            disabled=False,
        )
        
        self.dd_box = widgets.VBox([dd_root])
        self.selected_node = None
           
    def find_child_index(self, description):
        counter = 0
        for child in self.dd_box.children:
            if child.description == description:
                break
            counter += 1
        return counter
    
    def detect_change(self, change, node=client):
        if change['new'] == '-':
            index = self.find_child_index(change['owner'].description)
            self.dd_box.children = self.dd_box.children[:index+1]
        else:
            child_id = int(change['owner'].description.split('_')[-1][:-1])
            if child_id < len(self.dd_box.children)-1:
                self.dd_box.children = self.dd_box.children[:child_id+1]

            if node[change['new']].item['attributes']['structure_family'] == 'node':
                next_level = list(node[change['new']])
                next_level.insert(0, "-")
                dd_next_level = widgets.Dropdown(
                    options=next_level,
                    value=next_level[0],
                    description=f'Node_{child_id+1}:',
                    disabled=False,
                )
                self.dd_box.children += (dd_next_level,)
                dd_next_level.observe(functools.partial(self.detect_change, node=node[change['new']]), names='value')
            else:
                df_data = widgets.HTML(
                    value=node[change['new']].read().head().to_html(),
                    placeholder='Dataframe',
                    description='Dataframe:',
                )
                self.dd_box.children += (df_data,)
                self.selected_node = node[change['new']]
                
    def display_box(self):
        self.dd_box.children[0].observe(self.detect_change, names='value')
        display(self.dd_box)
        
    def get_selected_node(self):
        return self.selected_node

In [ ]:
# Run dynamic dropdown menus
dd_group = DropDownGroup(client)
dd_group.display_box()

In [ ]:
# Use this method to use the dataset that was selected from the dropdown menus previously
dd_node = dd_group.get_selected_node()
dd_node[["energy", 'i0']]

In [ ]:
dd_node.metadata

## Filtering rows with sliders

In [ ]:
# save the data in our local environment
df = dd_node.read()

preslit_data = df['Preslit']
energy_data = df['energy']

print(type(df))
print(df.shape)

In [ ]:
preslit_step = (max(preslit_data) - min(preslit_data))/(len(preslit_data)*2)
preslit_value = (max(preslit_data) - min(preslit_data)) * np.random.random() + min(preslit_data)
slider = widgets.FloatSlider(
    value=preslit_value,
    min=min(preslit_data),
    max=max(preslit_data),
    step=preslit_step,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

display(slider)

In [ ]:
energy_step = (max(energy_data) - min(energy_data))/(len(energy_data)*2)
energy_value_min = round(((max(energy_data) - min(energy_data))/2) * np.random.random() + min(energy_data),5)
energy_value_max = round(max(energy_data) - ((max(energy_data) - min(energy_data))/2) * np.random.random(),5)
range_slider = widgets.FloatRangeSlider(
    value=[energy_value_min, energy_value_max],
    min=min(energy_data),
    max=max(energy_data),
    step=energy_step,
    description='Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.5f',
)

min_limit = widgets.Text(
    value="0.0, 0.0",
    placeholder='Limits',
    description='Limits:',
    disabled=False
)

link = widgets.jslink((range_slider, 'value'), (min_limit, 'value'))

display(range_slider, min_limit)

In [ ]:
# A range slider can be used to define the max and min limits the filter applied to the dataframe
filtered_df = df[df['energy'].between(range_slider.value[0], range_slider.value[1])]
print(filtered_df.shape)
filtered_df

## Filtering columns with a selection box

In [ ]:
df_column_names = list(df.columns)
df_selection = widgets.SelectMultiple(
    options=df_column_names,
    value=[df_column_names[0]],
    #rows=10,
    description='dataframe',
    disabled=False
)

display(df_selection)

In [ ]:
xdi_df = df[list(df_selection.value)]
xdi_df

## Creating simple plots with selection boxes and dataframes

In [ ]:
column_list = list(df.columns)
x_axis = widgets.Dropdown(
    options=column_list,
    value=column_list[0],
    description='X axis:',
    disabled=False,
)

y_axis = widgets.Dropdown(
    options=column_list,
    value=column_list[0],
    description='Y axis:',
    disabled=False,
)

display(x_axis, y_axis)

In [ ]:
df.plot(x_axis.value, y_axis.value)

## More plots

In [ ]:
import matplotlib.pyplot as plt

from aimmdb.tree import RawMongoQuery as RawMongo

In [ ]:
fig, ax = plt.subplots()
ax.set_title(client["dataset"]["aimm_ncm"]["sample"]["h6bjpUSB4Qc"].metadata["sample"]["name"])

for k, v in client["dataset"]["aimm_ncm"]["sample"]["h6bjpUSB4Qc"]["uid"].search(RawMongo({"metadata.charge.cycle" : 0})).items():
    df = v.read()
    element = v.metadata["element"]["symbol"]
    edge = v.metadata["element"]["edge"]
    if "mutrans" in df.columns:
        df.plot("energy", "mutrans", ax=ax, label=f"{element}-{edge}")

In [ ]:
fig, ax = plt.subplots()
for k, v in client["dataset"]["aimm_ncm"]["sample"]["h6bjpUSB4Qc"]["element"]["Ni"]["edge"]["L3"]["uid"].items():
    charge = v.metadata["charge"]
    cycle, voltage, state = [charge[k] for k in ["cycle", "voltage", "state"]]
    df = v.read()
    df.plot("energy", "mu_tey", ax=ax, label=f"cycle {cycle} {voltage}V {state}")

## Search and queries

In [ ]:
client["uid"].search(RawMongo({"metadata.sample.dataset" : "aimm_ncm", "metadata.element.edge" : "K", "metadata.element.symbol" : "Ni"}))['kBZPKDsZHaQ']

In [ ]:
client["dataset"]["aimm_ncm"]["sample"]["h6bjpUSB4Qc"]["element"]["Ni"]["edge"]["L3"]["uid"].search(RawMongo({"metadata.charge.cycle" : 0})).values_indexer[0]

# 2. Write data

## Create array

In [ ]:
import io
import tifffile

In [ ]:
#Upload a tif file(s)
array_upload = widgets.FileUpload(
    accept='.tif',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)
display(array_upload)

In [ ]:
arr_list = []
for key, value in array_upload.value.items():
    tiff_array = tifffile.TiffFile(io.BytesIO(value["content"])).asarray()
    arr_list.append(tiff_array)

print(arr_list[0].shape)
print(arr_list[0])

In [ ]:
arr_meta_list = []
for i in range(len(arr_list)):
    array_metadata={"dataset" : "sandbox", "scan_id": i+1, "method": "A", "element":{"symbol": "Co", "edge": "K"}}
    arr_meta_list.append(array_metadata)
    
print(arr_meta_list[0])

## Create DataFrames

In [ ]:
import json

In [ ]:
#Upload a json file
df_upload = widgets.FileUpload(
    accept='.json',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)
display(df_upload)

In [ ]:
df_list = []
df_meta_list = []
counter = 1
for key, value in df_upload.value.items():
    file_data = json.loads(value["content"].decode('utf8'))
    dataframe_metadata = {}
    data = {}
    for item in file_data:
        if "Energy" in item:
            data = item.copy()
        else:
            if "element" in item:
                element = {"element":{"symbol": item["element"], "edge": item["edge"]}}
                item.pop("element")
                item.pop("edge")
                item.update(element)
            dataframe_metadata.update(item)
            
    json_dataframe = pandas.DataFrame(data)
    dataframe_metadata.update({"scan_id": counter, "method": "D"})
    
    df_list.append(json_dataframe)
    df_meta_list.append(dataframe_metadata)
    counter += 1

print(df_list[0].shape)
df_list[0].head()

In [ ]:
df_meta_list[0]

## Write an array

In [ ]:
array_key = client['uid'].write_array(arr_list[0], arr_meta_list[0])
array_key

## Write a dataframe

In [ ]:
df_key = client["uid"].write_dataframe(df_list[0], df_meta_list[0])
df_key

## Validate write/upload

In [ ]:
results = client["dataset"]["sandbox"].search(RawMongo({"metadata.element.symbol":"Co"})) # Search for all element that meet the search criteria
list(results["uid"])

### Validate array

In [ ]:
array_result = results["uid"][array_key].read() # Get first item of all results
results["uid"][array_key].metadata

In [ ]:
array_result

### Validate dataframe

In [ ]:
dataframe_result = results["uid"][df_key].read()
results["uid"][df_key].metadata

In [ ]:
dataframe_result